In [5]:
"""
Import statements
"""

from ipynb.fs.full.Extract_career_path import *
from ipynb.fs.full.Data_handling import *
from ipynb.fs.full.Competences import *

from collections import OrderedDict
import matplotlib.pyplot as pp

In [2]:
"""
Vacancies: add information about the job to the current vacancy 
"""

def extract_job_vac(vac, job_database, competence_relevance_database):
    vacancy_job = OrderedDict()

    for job in job_database.keys():
        if job == 'data scientist': 
            regular_expression = re.compile(r'data scientist | data science', re.IGNORECASE)
        elif job == 'data analyst':
            regular_expression = re.compile(r'data analyst | data analytics', re.IGNORECASE)
        else: 
            regular_expression = re.compile(job, re.IGNORECASE)
        regex_result = re.finditer(regular_expression, vac)

        for result in regex_result:
            index = job.capitalize()
            dspp = job_database[job]['DSPP']
            
            if dspp != 'NA':
                vacancy_job[index] = {'dspp': dspp}
                vacancy_job[index].update({'relevance_scores': competence_relevance_database[dspp]})
    
    return vacancy_job

job_database = load_jobs(JOB_DATA)
competence_relevance_database = load_dspp_mapping(COMPETENCE_RELEVANCE)


def search_for_job(vacancies): 
    vac_jobs = []
    for i in range(1, len(vacancies)+1): 
        first_check = ' '.join(norm_sentence(vacancies['title'][i]))
        info = extract_job_vac(first_check, job_database, competence_relevance_database)
        if info == {}: 
            next_check = ' '.join(list(itertools.chain.from_iterable(ast.literal_eval(df['description'][i]))))
            info = extract_job_vac(next_check, job_database, competence_relevance_database)
        if info != {}: 
            vac_jobs.append(list(info.items())[0])
        else: 
            vac_jobs.append(None)
    vacancies['Job info'] = vac_jobs
    return vacancies, vac_jobs
 
vac_job_extraction, list_of_sims = search_for_job(df)

In [3]:
# function that calculates the total vacancy job score
def vac_job_cfds(jobs, similarity): 
    products = []
    index = 0 
    for one in jobs['Job info']:
        dspp_sim = []
        if one == None: 
            # if there is no job found, the similarity score is stored 
            products.append(list(similarity[index]))
        else: 
            competences = one[1]['relevance_scores']
            for comp in one[1]['relevance_scores']: 
                if one[1]['relevance_scores'].get(comp) == 'NA':
                    dspp_sim.append(0) 
                else:
                    dspp_sim.append(int(one[1]['relevance_scores'].get(comp)))
            products.append([min((b/9)*a, 1) for a, b in zip(list(similarity[index]), dspp_sim)])
        index += 1 
    return products

# function that calculates the normalized total vacancy job score 
def vac_job_cfds_norm(jobs, similarity): 
    products = []
    index = 0 
    for one in jobs['Job info']:
        dspp_sim = []
        if one == None: 
            products.append(list(norm_vector(similarity[index], 0.25)))
        else: 
            competences = one[1]['relevance_scores']
            for comp in one[1]['relevance_scores']: 
                if one[1]['relevance_scores'].get(comp) == 'NA':
                    dspp_sim.append(0) # set to 1 when multiplying 
                else:
                    dspp_sim.append(int(one[1]['relevance_scores'].get(comp)))

            products.append([min(((b/9)*a)/0.2, 1) for a, b in zip(list(similarity[index]), dspp_sim)])
        index += 1 
    return products

job_cfds_scores_vac = vac_job_cfds(vac_job_extraction, vac_old_sim)
norm_job_scores = vac_job_cfds_norm(vac_job_extraction, vac_old_sim)

In [4]:
total_lines = []
for line in job_cfds_scores_vac:
    total_lines += line 